In [455]:
from bs4 import BeautifulSoup
import requests
from IPython.core.display import display, HTML
import numpy as np
import re
import pickle
import pandas as pd
import re

## Grab names for all NCAA teams

In [80]:
url = "https://www.sports-reference.com/cbb/schools/"
response = requests.get(url)
teams_text = response.text
team_soup = BeautifulSoup(teams_text, "lxml")
team_table = team_soup.find('table')
team_rows = team_table.find_all('tr')

In [250]:
#note that after every 20 teams, there is another header row
teams = {}
for i in [i for i in range(1,len(team_rows)) if i % 21 != 0]:
    items = team_rows[i].find_all('td')
    link = items[0].find('a')
    school, url = link.text, link['href']
    teams[school] = [url] + [i.text for i in items]

In [370]:
with open('teams_list.pickle', 'wb') as handle:
    pickle.dump(teams, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [381]:
#find teams in D1 for particular season
season = 2019
season_teams = []
for team in teams.keys():
    if int(teams[team][3]) <= season <= int(teams[team][4]):
        season_teams.append(team)


In [560]:
#create team_lookup for names - url name as key, values include name, formal name, and team url
team_lookup = {}
for key in season_teams:
    sched_url = "https://www.sports-reference.com" + teams[key][0] + str(season) +"-schedule.html"
    sched_response = requests.get(sched_url)
    sched_text = sched_response.text
    sched_soup = BeautifulSoup(sched_text,"lxml")
    sched_table = sched_soup.find('table')
    simple_name = re.split("\/",teams[key][0])[3]
    sched_rows = sched_table.find_all('tr')
    name = sched_table.find('a').text
    formal_name = teams[key][1]
    team_lookup[simple_name] = [name] + [formal_name] + [teams[key][0]]

In [564]:
with open('teams_lookup.pickle', 'wb') as handle:
    pickle.dump(team_lookup, handle, protocol=pickle.HIGHEST_PROTOCOL)

## GRAB gamelogs for each team for 2019

In [ ]:
gamelog = {}
for key in season_teams:
    game_url = "https://www.sports-reference.com" + teams[key][0] + str(season) +"-gamelogs.html"
    game_response = requests.get(game_url)
    gamelog_text = game_response.text
    gamelog_soup = BeautifulSoup(gamelog_text,"lxml")
    gamelog_table = gamelog_soup.find('table')
    gamelog_rows = gamelog_table.find_all('tr')
    team = re.split("\/",teams[key][0])[3]
    for i in [i for i in range(2,len(gamelog_rows)) if i % 22 != 0 and i % 23 != 0 ]:
        items = gamelog_rows[i].find_all('td')
        link = items[0].find('a')
        date, url = link.text, link['href']
        gamelog[url + team] = [url] + [team] + [i.text for i in items]

In [384]:
with open('gamelog.pickle', 'wb') as handle:
    pickle.dump(gamelog, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [427]:
games = pd.DataFrame.from_dict(gamelog, orient = "index")

In [441]:
columns = ['url','Team','Date','Court','Opponent','Result','Tm','Opp','FG','FGA','FG%','3P','3PA','3P%','FT','FTA',
           'FT%','ORB','TRB','AST','STL','BLK','TOV','PF','blank','FG_O','FGA_O','FG%_O','3P_O','3PA_O','3P%_O','FT_O',
           'FTA_O','FT%_O','RB_O','TRB_O','AST_O','STL_O','BLK_O','TOV_O','PF_O']
games.columns=columns
games.drop(columns='blank');

In [636]:
#formatting data frame
games['Date'] = pd.to_datetime(games['Date'])
num_cols_basic = ['Tm','Opp','FG','FGA','FG%','3P','3PA','3P%','FT','FTA','FT%','ORB','TRB','AST','STL','BLK',
                  'TOV','PF','blank','FG_O','FGA_O','FG%_O','3P_O','3PA_O','3P%_O','FT_O','FTA_O','FT%_O','RB_O',
                  'TRB_O','AST_O','STL_O','BLK_O','TOV_O','PF_O']
for i in num_cols_basic:
    games[i] = pd.to_numeric(games[i])

In [759]:
#create dictionary for results, which will be used for further calculations
r_keys = ['L', 'W', 'L\xa0(1 OT)', 'L\xa0(2 OT)', 'W\xa0(2 OT)','W\xa0(1 OT)', 'W\xa0(3 OT)', 'L\xa0(3 OT)']
r_wins = [0,1,0,0,1,1,1,0] #1 for win
r_OT = [0,0,1,2,2,1,3,3] #overtimes played
r_MP = [40,40,45,50,50,45,55,55] #minutes played

results_dict = dict(zip(r_keys,zip(r_wins,r_OT,r_MP)))

In [767]:
#adding columns to help with further data manipulation
games['Wins'] = [results_dict[x][0] for x in games.Result]
games['OT'] = [results_dict[x][1] for x in games.Result]
games['MP']= [results_dict[x][2] for x in games.Result]
games['PD'] = games['Tm']-games['Opp']

In [485]:
with open('advanced_gamelog.pickle', 'wb') as handle:
    pickle.dump(advanced_gamelog, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [487]:
advanced_games = pd.DataFrame.from_dict(advanced_gamelog, orient = "index")

In [540]:
columns = ['url','Team','Date','Court','Opponent','Result','Tm','Opp','ORtg','DRtg','Pace','FTr','3PAr','TS%','TRB%',
           'AST%','STL%','BLK%','blank1','OeFG%','OTOV%','ORB%','OFT/FGA','blank2','DeFG%','DTOV%','DRB%','DFT/FGA']
advanced_games.columns=columns
advanced_games.drop(columns=['blank1','blank2']);

In [640]:
advanced_games['Date'] = pd.to_datetime(advanced_games['Date'])
num_cols_advanced = ['Tm','Opp','ORtg','DRtg','Pace','FTr','3PAr','TS%','TRB%',
           'AST%','STL%','BLK%','blank1','OeFG%','OTOV%','ORB%','OFT/FGA','blank2','DeFG%','DTOV%','DRB%','DFT/FGA']
for i in num_cols_advanced:
    advanced_games[i] = pd.to_numeric(advanced_games[i])

In [773]:
#adding columns to help with further data manipulation
advanced_games['Wins'] = [results_dict[x][0] for x in advanced_games.Result]
advanced_games['OT'] = [results_dict[x][1] for x in advanced_games.Result]
advanced_games['MP']= [results_dict[x][2] for x in advanced_games.Result]
advanced_games['PD'] = advanced_games['Tm']-advanced_games['Opp']

## Preparing clean data file

In [641]:
# Final DF
df = pd.DataFrame(games['url'])
df['Team'] = [team_lookup[x][0] for x in games['Team']]
df['Date'] = games['Date']
df['Opponent'] = games['Opponent']
df['Court'] = games['Court']
df['PD'] = games['PD']

In [738]:
# GP = Games Played prior to this game
df['GP'] = games.groupby('Team')['Team'].transform(lambda x: x.expanding().count()-1)

In [749]:
# get prior average stats for each team before the game
for i in ['Tm','Opp','FG','FGA','FG%','3P','3PA','3P%','FT','FTA','FT%','ORB','TRB','AST','STL','BLK',
                  'TOV','PF','FG_O','FGA_O','FG%_O','3P_O','3PA_O','3P%_O','FT_O','FTA_O','FT%_O','RB_O',
                  'TRB_O','AST_O','STL_O','BLK_O','TOV_O','PF_O','Wins','OT','MP']:
    df[i] = games.groupby('Team')[i].transform(lambda x: x.expanding().sum()-x)/df['GP']

In [778]:
def data_manip(games,columns,team_lookup=team_lookup):
    '''
    this helps to clean data. update this here
    games = gamelog information by team ______
    columns = names of columns in the dataframe
    team_lookup = to get the right name
    
    output: dataframe cleaned!

    '''
    df = pd.DataFrame(games['url'])
    df['Team'] = [team_lookup[x][0] for x in games['Team']]
    df['Date'] = games['Date']
    df['Opponent'] = games['Opponent']
    df['Court'] = games['Court']
    df['PD'] = games['PD']
    df['GP'] = games.groupby('Team')['Team'].transform(lambda x: x.expanding().count()-1)
    for i in columns:
        df[i] = games.groupby('Team')[i].transform(lambda x: x.expanding().sum()-x)/df['GP']
    return df

In [779]:
adf = data_manip(advanced_games,num_cols_advanced)

In [780]:
adf.head()

,url,Team,Date,Opponent,Court,PD,GP,Tm,Opp,ORtg,...,blank1,OeFG%,OTOV%,ORB%,OFT/FGA,blank2,DeFG%,DTOV%,DRB%,DFT/FGA
/cbb/boxscores/2018-11-06-19-abilene-christian.htmlabilene-christian,/cbb/boxscores/2018-11-06-19-abilene-christian...,Abilene Christian,2018-11-06,Arlington Baptist,,53,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/cbb/boxscores/2018-11-09-20-abilene-christian.htmlabilene-christian,/cbb/boxscores/2018-11-09-20-abilene-christian...,Abilene Christian,2018-11-09,Arkansas State,,21,1.0,107.000000,54.000000,142.700000,...,NaN,0.705000,12.600,50.000000,0.212000,NaN,0.4180,27.700000,86.200000,0.26500
/cbb/boxscores/2018-11-15-21-denver.htmlabilene-christian,/cbb/boxscores/2018-11-15-21-denver.html,Abilene Christian,2018-11-15,Denver,@,6,2.0,100.500000,63.500000,132.400000,...,NaN,0.670500,16.550,44.650000,0.324000,NaN,0.4550,23.100000,80.600000,0.25950
/cbb/boxscores/2018-11-22-18-abilene-christian.htmlabilene-christian,/cbb/boxscores/2018-11-22-18-abilene-christian...,Abilene Christian,2018-11-22,Elon,N,16,3.0,89.333333,62.666667,124.866667,...,NaN,0.630667,15.600,35.933333,0.304333,NaN,0.4960,22.433333,77.066667,0.18600
/cbb/boxscores/2018-11-23-20-pacific.htmlabilene-christian,/cbb/boxscores/2018-11-23-20-pacific.html,Abilene Christian,2018-11-23,Pacific,@,2,4.0,85.000000,61.000000,125.800000,...,NaN,0.627000,14.025,34.350000,0.241750,NaN,0.5065,21.400000,77.600000,0.22075


## GRAB URL FOR ALL GAMES

## Grab data from single game - hold off on pulling individual game data for now

Input url for each game
Output data table

In [7]:
url = 'https://www.sports-reference.com/cbb/boxscores/2020-03-11-21-stanford.html'

box = requests.get(url)
boxscores = box.text
soup = BeautifulSoup(boxscores, "html5lib")

In [65]:
tables = soup.find_all('table')
#note this will grab 8 tables (the last 4 will be basic and advanced stats, for each team)

Pull Data from class "scorebox_meta": Date, Location <- this does not show up in a table

Pull Data from table "line-score": <- this also does not show up in a table...
Output: Away Team, Home Team, Away Score 1H, Away Score 2H, Away Score F, Home Score 1H, Home Score 2h, Home Score F

? What about overtimes...

## Scratchwork

In [583]:
#test case for gamelogs
cal_url = "https://www.sports-reference.com/cbb/schools/california/2019-gamelogs.html"
cal_response = requests.get(cal_url)
gamelog_text = cal_response.text
gamelog_soup = BeautifulSoup(gamelog_text,"lxml")
gamelog_table = gamelog_soup.find('table')
gamelog_rows = gamelog_table.find_all('tr')
#test case continued
cal_log = {}
team = 'california'
for i in [i for i in range(2,len(gamelog_rows)) if i % 22 != 0 and i % 23 != 0 ]:
    items = gamelog_rows[i].find_all('td')
    link = items[0].find('a')
    date, url = link.text, link['href']
    cal_log[url + team] = [url] + [team] + [i.text for i in items]